## Imports

In [5]:
from cgi import test
from optparse import Values
from sqlite3 import Row
from time import time
import tkinter as tk
from turtle import width
import pyvisa
import time
# import keyboard
import time
import cv2
from PIL import Image, ImageTk
from webbrowser import get
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import threading as th
import minimalmodbus
from skopt import Optimizer
from skopt.plots import plot_objective
from skopt.plots import plot_evaluations
# from simple_pid import PID
import os
import pandas as pd
import random
import serial


Device Connection


In [2]:
rm = pyvisa.ResourceManager()
print(rm.list_resources())

('ASRL1::INSTR', 'ASRL6::INSTR', 'ASRL7::INSTR', 'ASRL13::INSTR', 'ASRL17::INSTR', 'ASRL18::INSTR')


In [10]:
# temperature control

# baudrate_temp = 9600
# ser = serial.Serial('com20', baudrate_temp, timeout=1, rtscts=True, dsrdtr=True)
# # ser.open()
# # ser.close()

# com_hot = minimalmodbus.Instrument(ser, 1, minimalmodbus.MODE_RTU)
# com_hot.serial.baudrate = 9600
# print(com_hot)


com_hot = minimalmodbus.Instrument('COM17', 1, minimalmodbus.MODE_RTU)
a = dir(com_hot)
print(com_hot)
# print(a)
# print("---------------")
# print(dir(com_hot))

minimalmodbus.Instrument<id=0x15c1cf8a3d0, address=1, mode=rtu, close_port_after_each_call=False, precalculate_read_size=True, clear_buffers_before_each_transaction=True, handle_local_echo=False, debug=False, serial=Serial<id=0x15c1cf37ca0, open=True>(port='COM17', baudrate=19200, bytesize=8, parity='N', stopbits=1, timeout=0.05, xonxoff=False, rtscts=False, dsrdtr=False)>


Position of the Camera and Stage don't necessarily matter since this is just calibrating temperatures

In [7]:
#ASRL7 = Linear Stage
com_pxpy = rm.open_resource('ASRL7::INSTR')
com_pxpy.baud_rate = 921600 #fixed

#Turns on Motion Controller
com_pxpy.write('1MO')
com_pxpy.write('2MO')

def move_to(position_x, position_y):
    position_x = str(position_x)
    position_y = str(position_y)
    com_pxpy.write('1PA'+position_x)
    time.sleep(0.1)              
    com_pxpy.write('2PA'+position_y)
    time.sleep(0.1)

In [1]:
def read_T(com_hot):
  a = com_hot.read_float(528) #double check whether this returns the current "actual" temperature
  #or just the set point temperature --> i think it's actually the "actual" temperature
  return a

In [3]:
a = read_T(com_hot)

NameError: name 'com_hot' is not defined

## Functions

In [8]:
## Additional (wrapper) Functions

def read_T(com_hot):
  a = com_hot.read_float(528) #double check whether this returns the current "actual" temperature
  #or just the set point temperature --> i think it's actually the "actual" temperature
  return a
    
def write_T(com_hot, T):
  com_hot.write_float(548, float(T))
  return

def delay_func(T_prev, T_new, cool_scale = 5):
  """ given two temperatures, predict the time required to allow for temp to equilibriate ([=] s)
  t = (m*dT) + b; t_cool = t * cool_scale
  """
  heat_flag = (T_new > T_prev)
  m = 60
  b = 15
  dT = np.abs(T_new-T_prev)

  t = (m*dT) + b

  if heat_flag == False:
    t = t * cool_scale

  return t

######

#assume TC is some dict with entry 'curr_T', and 'reading' is returning the current values,
#while 'writing' is setting the current values
def read_current_Temp_dummy(Tdict, text, pFlag=1):
  A = Tdict['curr_T']
  if pFlag == 1:
    print(f"{text} temperature (C): ", A)

  return A

def set_current_Temp_dummy(Tdict,T, pFlag=1):
  T_float = float(T)
  Tdict['curr_T'] = T_float

  #function to accomodate for length required to heat stage - can modify (also assuming
  #that cooling takes same amount of time as heating for same dT)

  if pFlag == 1:
    print("Set temperature (C): ", T_float)

  Tdict['T_set_hist'].append(T_float)

  return Tdict

def external_measure_dummy(Tdict):
  #this would be where we actually use an external thermocouple to take a measurement
  err_scale = 5 #random number between 0 and err_scale
  measurement = Tdict['curr_T'] + (err_scale * np.random.rand())

  Tdict['T_meas_hist'].append(measurement)
  return Tdict

##########


# Real Testing

## Continuous external

In [ ]:
RT= 23
low_T = 30
high_T = 200
num_T = 20

#Temp values to test over
T_list = np.arange(low_T, high_T, num_T)
#idea: have external thermocouple hooked up to measure continuously: assume continual measurements possible
#(i.e. time step = relatively short, on the order of seconds)

#no need to make T_set since that corresponds to T_list
T_read = []

for i in range(num_T):
  write_T(T_list[i])

  if i == 0:
    t_sleep = delay_func(RT, T_list[i])
  else:
    t_sleep = delay_func(T_list[i-1], T_list[i])

  time.sleep(t_sleep)

  curr_T = read_T()
  T_read.append(curr_T)


In [ ]:
x = 0
y = 50
move_to(x,y)

#1(x): -49 -> 50
#2(y): -50 -> 50

In [ ]:
print(read_T(com_hot))

In [ ]:
import pylablib as pll

In [9]:
import serial

# Connect to the Omega temperature controller
ser = serial.Serial('com1', baudrate=9600, parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE, bytesize=serial.EIGHTBITS, timeout=1)

# Read the current temperature
ser.write(b'READ_TEMP')  # Send the command to read the temperature
temperature = float(ser.read(5).decode().strip())  # Read the temperature response
print(f"Current temperature: {temperature:.2f} °C")

# Set the temperature setpoint
setpoint_temperature = 50.0
ser.write(f'SET_TEMP {setpoint_temperature}'.encode())  # Send the command to set the temperature

ser.close()  # Close the serial connection

ValueError: could not convert string to float: ''